In [1]:
!kaggle competitions download -c titanic


  0%|          | 0.00/34.1k [00:00<?, ?B/s]
100%|##########| 34.1k/34.1k [00:00<00:00, 3.19MB/s]


In [2]:
!unzip titanic

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [14]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# ref : https://www.kaggle.com/code/rohank05/titanic-using-xgboost/notebook?scriptVersionId=116764193

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [15]:
def age_filler(data):
    data['initial'] = 0
    
    for i in data:
        data['initial']=data.Name.str.extract('([A-Za-z]+)\.')
        
    data['initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                            ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace=True)
    
    x = data.groupby('initial')['Age'].mean().round()
    
    data['Age'] = data.groupby("initial")['Age'].apply(lambda x: x.fillna(x.mean()))    
    
    return data['Age']

In [16]:
train['Age'] = age_filler(train)
test['Age'] = age_filler(train)

In [17]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,initial
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [18]:
def prepare_data(x, IsTrain=True):
    x['Age_Band'] = pd.cut(x['Age'], bins=[0, 16, 32, 48, 64, np.inf], labels=[0, 1, 2, 3, 4])
    x["Embarked"].fillna("unk", inplace=True)
    x["Sex"] = x["Sex"].map({"male":0, "female":1}).astype(int)
    x["Embarked"] = x["Embarked"].map({"S":0, "C":1, "Q":2})
    
    Pclass_encoded = pd.get_dummies(x["Pclass"], prefix="Pclass")
    Embarked_encoded = pd.get_dummies(x["Embarked"], prefix="Embarked")
    Age_band_encoded = pd.get_dummies(x["Age_Band"], prefix="AgeBand")
    SibSp_encoded = pd.get_dummies(x["SibSp"], prefix="SibSp")
    Parch_encoded = pd.get_dummies(x["Parch"], prefix="Parch")
    
    x.drop({"PassengerId","Name", "Ticket","Cabin","Fare","Pclass", "Embarked", "Age", "Age_Band", "SibSp", "Parch"}, axis=1, inplace=True)
    
    x = pd.concat([x, Pclass_encoded, Embarked_encoded, Age_band_encoded, SibSp_encoded, Parch_encoded], axis=1)
    if IsTrain:
        y = x["Survived"]
        x.drop("Survived", axis=1, inplace=True)
        return x, y
    else:
        return x

In [19]:
test_copy = test.copy()

x, y = prepare_data(train)
x_test = prepare_data(test_copy, False)

In [20]:
x.drop(["initial"], axis=1, inplace=True)
x.head()

,Sex,Pclass_1,Pclass_2,Pclass_3,Embarked_0.0,Embarked_1.0,Embarked_2.0,AgeBand_0,AgeBand_1,AgeBand_2,...,SibSp_4,SibSp_5,SibSp_8,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,0,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [21]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [22]:
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42,
                          learning_rate=0.01, n_estimators=68, verbosity=0)

In [23]:
model.fit(x_train, y_train)
prediction = model.predict(x_val)

score = accuracy_score(prediction, y_val)
score

C:\Users\SooM\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.8212290502793296

In [24]:
x_train

,Sex,Pclass_1,Pclass_2,Pclass_3,Embarked_0.0,Embarked_1.0,Embarked_2.0,AgeBand_0,AgeBand_1,AgeBand_2,...,SibSp_4,SibSp_5,SibSp_8,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
674,0,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
720,1,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
261,0,0,0,1,1,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
430,0,1,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
400,0,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
310,1,1,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
403,0,0,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
515,0,1,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [25]:
x_test

,Sex,Pclass_1,Pclass_2,Pclass_3,Embarked_0,Embarked_1,Embarked_2,AgeBand_0,AgeBand_1,AgeBand_2,...,SibSp_5,SibSp_8,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9
0,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
414,1,1,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
415,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
416,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [26]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=68, n_jobs=12, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=0)

In [27]:
x_val

,Sex,Pclass_1,Pclass_2,Pclass_3,Embarked_0.0,Embarked_1.0,Embarked_2.0,AgeBand_0,AgeBand_1,AgeBand_2,...,SibSp_4,SibSp_5,SibSp_8,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
735,0,0,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
411,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
163,0,0,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
380,1,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
853,1,1,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,0,1,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
244,0,0,0,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
692,0,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
728,0,0,1,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [29]:
y_pred = model.predict(x_train)

In [30]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_train, y_pred)
score

0.8384831460674157

In [31]:
y_pred = model.predict(x_test)
y_pred = np.array(y_pred)
y_pred = y_pred.round().astype(int)

ValueError: Feature shape mismatch, expected: 26, got 418

In [ ]:
out_df = pd.DataFrame(data={'PassengerId': test.PassengerId.values, 'Survived': y_pred.flatten()})
out_df.to_csv('submission.csv', index=False)